In [8]:
import os
import pandas as pd
from collections import defaultdict
from functools import reduce

In [17]:
GSE = "Cell_GSE57249"
path = "/root/userfolder/quantify/"+GSE
SRR = pd.read_csv(os.path.join(path, "SRR.list"),header=None)[0].values.tolist()

# key: sample, value: {geneid:readcounts}
transcript = defaultdict(dict)
gene = defaultdict(dict)

for sample in SRR:
    genefile = os.path.join(path, "iREAD/RSEM", sample+"_res", sample+".genes.results")
    tranfile = os.path.join(path, "iREAD/RSEM", sample+"_res", sample+".isoforms.results")
    gene_pd = pd.read_csv(genefile, sep="\t")[["gene_id", "TPM"]].values.tolist()
    tran_pd = pd.read_csv(tranfile, sep="\t")[["transcript_id", "TPM"]].values.tolist()
    gene[sample] = dict(zip([i[0] for i in gene_pd], [i[1] for i in gene_pd]))
    transcript[sample] = dict(zip([i[0] for i in tran_pd], [i[1] for i in tran_pd]))

In [18]:
#### get gene read counts
intersect_genes = defaultdict(list)
for sample,value in gene.items():
    intersect_genes[sample] = list(value.keys())
allgene = list(intersect_genes.values())
unions = reduce(lambda a,b: list(set(a).union(b)), allgene)
gene_read_counts = {}
for s in SRR:
    value = []
    for g in unions:
        if g not in gene[s]:
            value.append(0)
        else:
            value.append(gene[s][g])
    gene_read_counts[s] = value
gene_read_counts = pd.DataFrame(gene_read_counts)
gene_read_counts.rename(index=dict(zip(list(range(0,gene_read_counts.shape[0])), unions)),inplace=True)
gene_read_counts.to_csv(GSE+"_IR_gene_TPM.txt",sep="\t")

In [19]:
#### get transcript read counts
intersect = defaultdict(list)
for sample,value in transcript.items():
    intersect[sample] = list(value.keys())
allid = list(intersect.values())
unions = reduce(lambda a,b: list(set(a).union(b)), allid)
read_counts = {}
for s in SRR:
    value = []
    for u in unions:
        if u not in transcript[s]:
            value.append(0)
        else:
            value.append(transcript[s][u])
    read_counts[s] = value
read_counts = pd.DataFrame(read_counts)
read_counts.rename(index=dict(zip(list(range(0,read_counts.shape[0])), unions)),inplace=True)
read_counts.to_csv(GSE+"_IR_transcript_TPM.txt",sep="\t")